In [1]:
import glob
import pandas as pd
import xml.etree.ElementTree as ET
from datetime import datetime

In [2]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/source.zip

--2022-05-15 17:22:16--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/source.zip
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2707 (2.6K) [application/zip]
Saving to: ‘source.zip’

source.zip          100%[===================>]   2.64K  --.-KB/s    in 0s      

2022-05-15 17:22:17 (552 MB/s) - ‘source.zip’ saved [2707/2707]



In [3]:
!unzip source.zip

Archive:  source.zip
  inflating: source3.json            
  inflating: source1.csv             
  inflating: source2.csv             
  inflating: source3.csv             
  inflating: source1.json            
  inflating: source2.json            
  inflating: source1.xml             
  inflating: source2.xml             
  inflating: source3.xml             


In [4]:
tmpfile = "temp.tmp"
logfile = "logfile.txt"
targetfile = "transformed_data.csv"

In [5]:
def extract_from_csv(file_to_process):
  dataframe = pd.read_csv(file_to_process)
  return dataframe

In [6]:
def extract_from_json(file_to_process):
  dataframe = pd.read_json(file_to_process, lines = True)
  return dataframe

In [7]:
def extract_from_xml(file_to_process):
  dataframe = pd.DataFrame(columns = ["name", "height", "weight"])
  tree = ET.parse(file_to_process)
  root = tree.getroot()
  for person in root:
    name = person.find("name").text
    height = float(person.find("height").text)
    weight = float(person.find("weight").text)
    dataframe = dataframe.append({"name" : name, "height" : height, "weight" : weight}, ignore_index = True)
  
  return dataframe

In [15]:
def extract():
  extracted_data = pd.DataFrame(columns = ["name", "height", "weight"])   #create an empty dataframe

  for csvfile in glob.glob("*.csv"):
    extracted_data = extracted_data.append(extract_from_csv(csvfile), ignore_index = True)

  for jsonfile in glob.glob("*.json"):
    extracted_data = extracted_data.append(extract_from_json(jsonfile), ignore_index = True)
  
  for xmlfile in glob.glob("*.xml"):
    extracted_data = extracted_data.append(extract_from_xml(xmlfile), ignore_index = True)
  
  return extracted_data

In [16]:
def transform(data):

  data["height"] = round(data.height * 0.0254, 2)

  data["weight"] = round(data.weight * 0.45359237, 2)

  return data

In [17]:
def load(targetfile, data_to_load):
  data_to_load.to_csv(targetfile)

In [18]:
def log(message):
  timestamp_format = "%Y-%h-%D-%h:%m:%s"
  now = datetime.now()
  timestamp = now.strftime(timestamp_format)
  with open("logfile.txt", "a") as f:
    f.write(timestamp + "," + message + "\n")

In [19]:
log("ETL job started")

In [20]:
log("Extract phase Started")
extracted_data = extract()
log("Extract phase Ended")
extracted_data

,name,height,weight
0,alex,65.78,112.99
1,ajay,71.52,136.49
2,alice,69.40,153.03
3,ravi,68.22,142.34
4,joe,67.79,144.30
5,alex,65.78,112.99
6,ajay,71.52,136.49
7,alice,69.40,153.03
8,ravi,68.22,142.34
9,joe,67.79,144.30


In [21]:
log("Transform phase Started")
transformed_data = transform(extracted_data)
log("Transform phase Ended")
transformed_data

,name,height,weight
0,alex,1.67,51.25
1,ajay,1.82,61.91
2,alice,1.76,69.41
3,ravi,1.73,64.56
4,joe,1.72,65.45
5,alex,1.67,51.25
6,ajay,1.82,61.91
7,alice,1.76,69.41
8,ravi,1.73,64.56
9,joe,1.72,65.45


In [22]:
log("Load phase Started")
load(targetfile, transformed_data)
log("Load phase Ended")

In [23]:
log("ETL job ended")

In [26]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip


--2022-05-15 17:54:52--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4249 (4.1K) [application/zip]
Saving to: ‘datasource.zip’

datasource.zip      100%[===================>]   4.15K  --.-KB/s    in 0s      

2022-05-15 17:54:52 (321 MB/s) - ‘datasource.zip’ saved [4249/4249]



In [27]:
!unzip datasource.zip -d dealership_data

Archive:  datasource.zip
  inflating: dealership_data/used_car_prices1.csv  
  inflating: dealership_data/used_car_prices2.csv  
  inflating: dealership_data/used_car_prices3.csv  
  inflating: dealership_data/used_car_prices1.json  
  inflating: dealership_data/used_car_prices2.json  
  inflating: dealership_data/used_car_prices3.json  
  inflating: dealership_data/used_car_prices1.xml  
  inflating: dealership_data/used_car_prices2.xml  
  inflating: dealership_data/used_car_prices3.xml  


In [28]:
tmpfile = "dealership_temp.tmp"
logfile = "dealership_logfile.txt"
targetfile = "dealership_transformed_data.csv"

In [29]:
def extract_from_csv1(file_to_process):
  dataframe = pd.read_csv(file_to_process)
  return dataframe

In [54]:
def extract_from_json1(file_to_process):
  dataframe = pd.read_json(file_to_process, lines = True)
  return dataframe

In [41]:
def extract_from_xml1(file_to_process):
  dataframe = pd.DataFrame(columns = ["car_model", "year_of_manufacture", "price", "fuel"])
  tree = ET.parse(file_to_process)
  root = tree.getroot()
  for person in root:
    car_model = person.find("car_model").text
    year_of_manufacture = int(person.find("year_of_manufacture").text)
    price = float(person.find("price").text)
    fuel = person.find("fuel").text
    dataframe = dataframe.append({"car_model" : car_model, "year_of_manufacture" : year_of_manufacture, "price" : price, "fuel" : fuel}, ignore_index = True)
  
  return dataframe

In [55]:
def extract1():
  extracted_data = pd.DataFrame(columns = ["car_model", "year_of_manufacture", "price", "fuel"])
  
  for csvfile in glob.glob("dealership_data/*.csv"):
    extracted_data = extracted_data.append(extract_from_csv1(csvfile), ignore_index = True)
  
  for jsonfile in glob.glob("dealership_data/*.json"):
    extracted_data = extracted_data.append(extract_from_json1(jsonfile), ignore_index = True)
  
  for xmlfile in glob.glob("dealership_data/*.xml"):
    extracted_data = extracted_data.append(extract_from_xml1(xmlfile), ignore_index = True)
  
  return extracted_data


In [56]:
def transform1(data):
  data["price"] = round(data.price, 2)

  return data

In [63]:
def load1(targetfile, data_to_load):
  data_to_load.to_csv(targetfile)

In [58]:
def log(message):
  timestamp_format = "%H:%M:%S-%h-%d-%Y"
  now = datetime.now()
  timestamp = now.strftime(timestamp_format)
  with open("dealership_logfile.txt", "a") as f:
    f.write(timestamp + "," + message + "\n")

In [59]:
log("ETL job started")
log("Extract phase started")
extracted_data = extract1()
log("Extract phase Ended")
extracted_data

,car_model,year_of_manufacture,price,fuel
0,alto k10,2016,4253.731343,Petrol
1,ignis,2017,7313.432836,Petrol
2,sx4,2011,6567.164179,Petrol
3,alto k10,2014,3731.343284,Petrol
4,wagon r,2013,4328.358209,Petrol
...,...,...,...,...
85,etios g,2015,5895.522388,Petrol
86,corolla altis,2013,8208.955224,Petrol
87,corolla,2004,2238.805970,Petrol
88,corolla altis,2010,7835.820896,Petrol


In [64]:
log("ETL job started")

log("Extract phase started")
extracted_data = extract1()
log("Extract phase ended")

log("Transform phase started")
transformed_data = transform1(extracted_data)
log("Transform phase ended")

log("Load phase started")
load1(targetfile, transformed_data)
log("Load phase ended")

log("ETL job ended")